In [1]:
import pandas as pd
import numpy as np

In [2]:
data_df = pd.read_csv('C:/Users/chanyoung/Desktop/RBFfitting/data/linear_SEM/linear_random_15.csv')

In [10]:
empty_row

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
empty_row

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
empty_row = pd.DataFrame(np.zeros((1, 15)), columns=data_df.columns)

In [48]:
data_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.394290,0.346584,2.283581,-1.559172,0.641352,-2.573262,-0.407517,2.417698,-1.459789,3.592460,-4.019680,0.790601,-1.833108,3.480442,6.263829
1,1.442065,1.753741,-0.565542,-0.019844,-0.217345,-0.505208,-0.836395,3.096807,0.408895,-0.085744,-0.584852,-2.247291,0.499501,-1.568848,-1.472852
2,0.158795,-2.018488,1.363381,0.369942,-0.553480,-0.829095,0.797979,-0.601401,3.293327,-1.130796,2.697072,-1.542107,5.722812,-1.352311,-10.048166
3,1.180820,-0.302838,2.811929,-1.769756,1.171722,-4.010535,-0.339541,0.686395,-2.497994,-1.055499,-0.307648,-3.129533,2.950652,1.869410,1.805169
4,-0.535161,-1.304216,0.194147,-0.127602,-0.704615,1.089379,-0.192465,-2.312792,1.273764,1.341614,0.833310,0.889260,0.310092,-0.762782,-5.504899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.840972,0.419546,-0.149228,-2.066541,-1.074929,-2.850332,0.213039,-0.793650,1.961616,-0.598479,2.261300,-1.971938,4.682809,-3.231114,-8.642133
96,1.944198,-0.182270,2.034609,-1.864585,2.520702,-3.416261,-1.658307,2.704336,-2.603829,0.000127,0.012510,-3.950297,2.483015,-2.110721,-1.172979
97,-0.344352,0.441910,-0.655871,0.063072,-1.891197,-0.224493,-1.741030,0.773491,2.123364,1.707830,-3.144857,1.326751,-2.125668,0.281651,-1.305697
98,2.452951,-0.174916,1.877249,-0.442922,0.942239,-1.480626,-0.138366,2.849003,0.308466,-0.692214,0.693084,-3.875676,2.494982,-0.641738,-2.852213


In [45]:
for i in range(len(data_df)):
    if i == 0:
        con_df = pd.concat([data_df[i:i+1], empty_row])
    else:
        con_df2 = pd.concat([data_df[i:i+1], empty_row])
        con_df = pd.concat([con_df, con_df2])

In [52]:
con_df.reset_index(drop= True,inplace = True)

In [54]:
con_df.to_csv('./data/GAIN_200.csv', index = False)

In [4]:
#%% Packages
import torch
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F

use_gpu = False  # set it to True to use GPU and False to use CPU
if use_gpu:
    torch.cuda.set_device(0)

In [66]:
#%% System Parameters
# 1. Mini batch size
mb_size = 128
# 2. Missing rate
p_miss = 0.5
# 3. Hint rate
p_hint = 0.9
# 4. Loss Hyperparameters
alpha = 10
# 5. Train Rate
train_rate = 0.8

#%% Data

# Data generation
Data = con_df.values

# Parameters
No = len(Data)
Dim = len(Data[0,:])

# Hidden state dimensions
H_Dim1 = Dim
H_Dim2 = Dim

# Normalization (0 to 1)
Min_Val = np.zeros(Dim)
Max_Val = np.zeros(Dim)

for i in range(Dim):
    Min_Val[i] = np.min(Data[:,i])
    Data[:,i] = Data[:,i] - np.min(Data[:,i])
    Max_Val[i] = np.max(Data[:,i])
    Data[:,i] = Data[:,i] / (np.max(Data[:,i]) + 1e-6)    

#%% Missing introducing
p_miss_vec = p_miss * np.ones((Dim,1)) 
   
Missing = np.zeros((No,Dim))

for i in range(Dim):
    A = np.random.uniform(0., 1., size = [len(Data),])
    B = A > p_miss_vec[i]
    Missing[:,i] = 1.*B

    
#%% Train Test Division    
   
idx = np.random.permutation(No)

Train_No = int(No * train_rate)
Test_No = No - Train_No
    
# Train / Test Features
trainX = Data[idx[:Train_No],:]
testX = Data[idx[Train_No:],:]

# Train / Test Missing Indicators
trainM = Missing[idx[:Train_No],:]
testM = Missing[idx[Train_No:],:]

#%% Necessary Functions

# 1. Xavier Initialization Definition
# def xavier_init(size):
#     in_dim = size[0]
#     xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
#     return tf.random_normal(shape = size, stddev = xavier_stddev)
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return np.random.normal(size = size, scale = xavier_stddev)
    
# Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C

In [77]:
augmentationX = Data
augmentationM = np.zeros_like(Data)
for i in range(Dim):
    A = np.random.uniform(0., 1., size = [len(Data),])
    B = A > p_miss_vec[i]
    augmentationM[:,i] = 1.*B

In [67]:
if use_gpu is True:
    D_W1 = torch.tensor(xavier_init([Dim*2, H_Dim1]),requires_grad=True, device="cuda")     # Data + Hint as inputs
    D_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True, device="cuda")

    D_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2]),requires_grad=True, device="cuda")
    D_b2 = torch.tensor(np.zeros(shape = [H_Dim2]),requires_grad=True, device="cuda")

    D_W3 = torch.tensor(xavier_init([H_Dim2, Dim]),requires_grad=True, device="cuda")
    D_b3 = torch.tensor(np.zeros(shape = [Dim]),requires_grad=True, device="cuda")       # Output is multi-variate
else:
    D_W1 = torch.tensor(xavier_init([Dim*2, H_Dim1]),requires_grad=True)     # Data + Hint as inputs
    D_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True)

    D_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2]),requires_grad=True)
    D_b2 = torch.tensor(np.zeros(shape = [H_Dim2]),requires_grad=True)

    D_W3 = torch.tensor(xavier_init([H_Dim2, Dim]),requires_grad=True)
    D_b3 = torch.tensor(np.zeros(shape = [Dim]),requires_grad=True)       # Output is multi-variate

theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]

#%% 2. Generator
if use_gpu is True:
    G_W1 = torch.tensor(xavier_init([Dim*2, H_Dim1]),requires_grad=True, device="cuda")     # Data + Mask as inputs (Random Noises are in Missing Components)
    G_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True, device="cuda")

    G_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2]),requires_grad=True, device="cuda")
    G_b2 = torch.tensor(np.zeros(shape = [H_Dim2]),requires_grad=True, device="cuda")

    G_W3 = torch.tensor(xavier_init([H_Dim2, Dim]),requires_grad=True, device="cuda")
    G_b3 = torch.tensor(np.zeros(shape = [Dim]),requires_grad=True, device="cuda")
else:
    G_W1 = torch.tensor(xavier_init([Dim*2, H_Dim1]),requires_grad=True)     # Data + Mask as inputs (Random Noises are in Missing Components)
    G_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True)

    G_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2]),requires_grad=True)
    G_b2 = torch.tensor(np.zeros(shape = [H_Dim2]),requires_grad=True)

    G_W3 = torch.tensor(xavier_init([H_Dim2, Dim]),requires_grad=True)
    G_b3 = torch.tensor(np.zeros(shape = [Dim]),requires_grad=True)
theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]

In [68]:
#%% 1. Generator
def generator(new_x,m):
    inputs = torch.cat(dim = 1, tensors = [new_x,m])  # Mask + Data Concatenate
    G_h1 = F.relu(torch.matmul(inputs, G_W1) + G_b1)
    G_h2 = F.relu(torch.matmul(G_h1, G_W2) + G_b2)   
    G_prob = torch.sigmoid(torch.matmul(G_h2, G_W3) + G_b3) # [0,1] normalized Output
    
    return G_prob

#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = torch.cat(dim = 1, tensors = [new_x,h])  # Hint + Data Concatenate
    D_h1 = F.relu(torch.matmul(inputs, D_W1) + D_b1)  
    D_h2 = F.relu(torch.matmul(D_h1, D_W2) + D_b2)
    D_logit = torch.matmul(D_h2, D_W3) + D_b3
    D_prob = torch.sigmoid(D_logit)  # [0,1] Probability Output
    
    return D_prob

#%% 3. Other functions
# Random sample generator for Z
def sample_Z(m, n):
    return np.random.uniform(0., 0.01, size = [m, n])        

# Mini-batch generation
def sample_idx(m, n):
    A = np.random.permutation(m)
    idx = A[:n]
    return idx

In [69]:
def discriminator_loss(M, New_X, H):
    # Generator
    G_sample = generator(New_X,M)
    # Combine with original data
    Hat_New_X = New_X * M + G_sample * (1-M)

    # Discriminator
    D_prob = discriminator(Hat_New_X, H)

    #%% Loss
    D_loss = -torch.mean(M * torch.log(D_prob + 1e-8) + (1-M) * torch.log(1. - D_prob + 1e-8))
    return D_loss

def generator_loss(X, M, New_X, H):
    #%% Structure
    # Generator
    G_sample = generator(New_X,M)

    # Combine with original data
    Hat_New_X = New_X * M + G_sample * (1-M)

    # Discriminator
    D_prob = discriminator(Hat_New_X, H)

    #%% Loss
    G_loss1 = -torch.mean((1-M) * torch.log(D_prob + 1e-8))
    MSE_train_loss = torch.mean((M * New_X - M * G_sample)**2) / torch.mean(M)

    G_loss = G_loss1 + alpha * MSE_train_loss 

    #%% MSE Performance metric
    MSE_test_loss = torch.mean(((1-M) * X - (1-M)*G_sample)**2) / torch.mean(1-M)
    return G_loss, MSE_train_loss, MSE_test_loss
    
def test_loss(X, M, New_X):
    #%% Structure
    # Generator
    G_sample = generator(New_X,M)

    #%% MSE Performance metric
    MSE_test_loss = torch.mean(((1-M) * X - (1-M)*G_sample)**2) / torch.mean(1-M)
    return MSE_test_loss, G_sample

In [70]:
optimizer_D = torch.optim.Adam(params=theta_D)
optimizer_G = torch.optim.Adam(params=theta_G)

In [71]:
#%% Start Iterations
for it in tqdm(range(500)):    
    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]  
    
    Z_mb = sample_Z(mb_size, Dim) 
    M_mb = trainM[mb_idx,:]  
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    if use_gpu is True:
        X_mb = torch.tensor(X_mb, device="cuda")
        M_mb = torch.tensor(M_mb, device="cuda")
        H_mb = torch.tensor(H_mb, device="cuda")
        New_X_mb = torch.tensor(New_X_mb, device="cuda")
    else:
        X_mb = torch.tensor(X_mb)
        M_mb = torch.tensor(M_mb)
        H_mb = torch.tensor(H_mb)
        New_X_mb = torch.tensor(New_X_mb)
    
    optimizer_D.zero_grad()
    D_loss_curr = discriminator_loss(M=M_mb, New_X=New_X_mb, H=H_mb)
    D_loss_curr.backward()
    optimizer_D.step()
    
    optimizer_G.zero_grad()
    G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = generator_loss(X=X_mb, M=M_mb, New_X=New_X_mb, H=H_mb)
    G_loss_curr.backward()
    optimizer_G.step()    
        
    #%% Intermediate Losses
    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('Train_loss: {:.4}'.format(np.sqrt(MSE_train_loss_curr.item())))
        print('Test_loss: {:.4}'.format(np.sqrt(MSE_test_loss_curr.item())))
        print()

  5%|████▏                                                                           | 26/500 [00:00<00:01, 247.70it/s]

Iter: 0
Train_loss: 0.3039
Test_loss: 0.289



 26%|████████████████████▍                                                          | 129/500 [00:00<00:01, 210.92it/s]

Iter: 100
Train_loss: 0.1466
Test_loss: 0.1695



 48%|█████████████████████████████████████▊                                         | 239/500 [00:01<00:01, 202.46it/s]

...

 64%|██████████████████████████████████████████████████▍                            | 319/500 [00:01<00:01, 166.47it/s]

Iter: 300
Train_loss: 0.1187
Test_loss: 0.1538



 83%|█████████████████████████████████████████████████████████████████▌             | 415/500 [00:02<00:00, 138.06it/s]

Iter: 400
Train_loss: 0.1174
Test_loss: 0.1653



100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 172.05it/s]


In [83]:
augmentationX = Data
augmentationM = np.zeros_like(Data)
for i in range(Dim):
    A = np.random.uniform(0., 1., size = [len(Data),])
    B = A > p_miss_vec[i]
    augmentationM[:,i] = 1.*B
Z_mb = sample_Z(200, 15) 

In [84]:
# Z_mb = sample_Z(Test_No, Dim) 
M_mb = augmentationM
X_mb = augmentationX
        
New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce

if use_gpu is True:
    X_mb = torch.tensor(X_mb, device='cuda')
    M_mb = torch.tensor(M_mb, device='cuda')
    New_X_mb = torch.tensor(New_X_mb, device='cuda')
else:
    X_mb = torch.tensor(X_mb)
    M_mb = torch.tensor(M_mb)
    New_X_mb = torch.tensor(New_X_mb)
    
MSE_final, Sample = test_loss(X=X_mb, M=M_mb, New_X=New_X_mb)
# np.set_printoptions(formatter={'float': lambda x: "{0:0.8f}".format(x)})

In [85]:
imputed_data = M_mb * X_mb + (1-M_mb) * Sample

In [97]:
imputed_df = pd.DataFrame(imputed_data.detach().numpy(), columns = con_df.columns)

In [100]:
imputed_df.to_csv('./data/augmentation_linear_200.csv', index = False)